In [ ]:
pip install -q tfds-nightly tensorflow matplotlib

     |████████████████████████████████| 4.0 MB 5.2 MB/s 


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import PIL
from PIL import Image

In [ ]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K

class MiniVGGNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# first CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# second CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(512))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))

		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

In [ ]:
model = MiniVGGNet.build(width=64, height=64, depth=3, classes=3)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 64, 64, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0

In [ ]:
datasets, info = tfds.load('beans', with_info=True, as_supervised=True)
beans_train, beans_test = datasets['train'], datasets['test']

print(info)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1034 [00:00<?, ? examples/s]

Shuffling beans-train.tfrecord...:   0%|          | 0/1034 [00:00<?, ? examples/s]

Generating validation examples...:   0%|          | 0/133 [00:00<?, ? examples/s]

Shuffling beans-validation.tfrecord...:   0%|          | 0/133 [00:00<?, ? examples/s]

Generating test examples...:   0%|          | 0/128 [00:00<?, ? examples/s]

Shuffling beans-test.tfrecord...:   0%|          | 0/128 [00:00<?, ? examples/s]

Dataset beans downloaded and prepared to /root/tensorflow_datasets/beans/0.1.0. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='beans',
    full_name='beans/0.1.0',
    description="""
    Beans is a dataset of images of beans taken in the field using smartphone
    cameras. It consists of 3 classes: 2 disease classes and the healthy class.
    Diseases depicted include Angular Leaf Spot and Bean Rust. Data was annotated
    by experts from the National Crops Resources Research Institute (NaCRRI) in
    Uganda and collected by the Makerere AI research lab.
    """,
    homepage='https://github.com/AI-Lab-Makerere/ibean/',
    data_path='/root/tensorflow_datasets/beans/0.1.0',
    download_size=171.69 MiB,
    dataset_size=171.63 MiB,
    features=FeaturesDict({
        'image': Image(shape=(500, 500, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=3),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=Fal

In [ ]:
%%capture
!pip install wandb

In [ ]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import matplotlib

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt
import numpy as np
import wandb
from wandb.keras import WandbCallback

def augmentation(image, label):
  image = tf.image.stateless_random_flip_left_right(image, seed=(2,3))
  image = tf.image.stateless_random_flip_up_down(image, seed=(2,3))
  image = tf.image.rot90(image)
  return image, label

def resize(image, label):
    return tf.image.resize(image, (64,64)), label

beans_train = beans_train.map(augmentation)
beans_test = beans_test.map(augmentation)

beans_train = beans_train.map(resize)
beans_test = beans_test.map(resize)

train_x = list(map(lambda x: x[0], beans_train))
train_y = list(map(lambda x: x[1], beans_train))

test_x = list(map(lambda x: x[0], beans_test))
test_y = list(map(lambda x: x[1], beans_test))

train_x = np.array(train_x) 
train_y = np.array(train_y)

test_x = np.array(test_x) 
test_y = np.array(test_y)

train_x = train_x.astype("float") / 255.0
test_x = test_x.astype("float") / 255.0

lb = LabelBinarizer()
train_y = lb.fit_transform(train_y)
test_y = lb.transform(test_y)

labelNames = ["angular_leaf_spot", "bean_rust", "healthy"]

print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

(1034, 64, 64, 3)
(128, 64, 64, 3)
(1034, 3)
(128, 3)


In [ ]:
# Set an experiment name to group training and evaluation
experiment_name = wandb.util.generate_id()

# setup wandb
wandb.init(project="beans4_VGG", 
           group=experiment_name,
           config={
               "epoch": 25,
               "batch_size": 128,
           })
config = wandb.config

In [ ]:
%%wandb

# initialize the optimizer and model
print("[INFO] compiling model...")

# An decay parameter was used. This argument is used to slowly reduce the learning rate over time.
# Rate Schedulers, decaying the learning rate is helpful in reducing overfitting
# and obtaining higher classification accuracy – the smaller the learning rate is, 
# the smaller the weight updates will be. A common setting for decay is to divide
# the initial learning rate by the total number of epochs – in this case, 
# we’ll be training our network for a total of 40 epochs with an initial learning rate of 0.01,
# therefore decay = 0.01 / 40.

opt = SGD(learning_rate=0.01, decay=0.01/25, momentum=0.9, nesterov=True)
model = MiniVGGNet.build(width=64, height=64, depth=3, classes=3)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

# train the network
print("[INFO] training network...")
H = model.fit(train_x, train_y, validation_data=(test_x, test_y),
              batch_size=config.batch_size, 
              epochs=config.epoch,
              verbose=1,
              callbacks=[WandbCallback()])
wandb.finish()

[INFO] compiling model...
[INFO] training network...
Epoch 1/25
9/9 [==============================] - 29s 3s/step - loss: 1.2891 - accuracy: 0.5948 - val_loss: 1.4231 - val_accuracy: 0.3203
Epoch 2/25
9/9 [==============================] - 28s 3s/step - loss: 1.2902 - accuracy: 0.5986 - val_loss: 4.9131 - val_accuracy: 0.3438
Epoch 3/25
9/9 [==============================] - 27s 3s/step - loss: 0.9049 - accuracy: 0.6779 - val_loss: 7.7641 - val_accuracy: 0.3281
Epoch 4/25
9/9 [==============================] - 28s 3s/step - loss: 0.7673 - accuracy: 0.7166 - val_loss: 6.4506 - val_accuracy: 0.3281
Epoch 5/25
9/9 [==============================] - 27s 3s/step - loss: 0.6238 - accuracy: 0.7795 - val_loss: 3.6354 - val_accuracy: 0.3359
Epoch 6/25
9/9 [==============================] - 30s 3s/step - loss: 0.4733 - accuracy: 0.8124 - val_loss: 2.4322 - val_accuracy: 0.3281
Epoch 7/25
9/9 [==============================] - 28s 3s/step - loss: 0.4414 - accuracy: 0.8404 - val_loss: 1.4261 - va

epoch,24
loss,0.2142
accuracy,0.91779
val_loss,2.92344
val_accuracy,0.46094
_runtime,699
_timestamp,1631487700
_step,24
best_val_loss,1.42306
best_epoch,0


epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,██▅▅▄▃▂▂▂▂▃▂▁▁▁▁▁▁▂▂▂▁▁▁▁
accuracy,▁▁▃▄▅▆▆▆▇▇▆▇██▇███▇▇▇██▇█
val_loss,▁▅█▇▃▂▁▁▂▃▅▃▄▂▁▂▂▇▃▄▂▂▁▂▃
val_accuracy,▁▂▁▁▁▁▆▆▃▃▂▅▃▄▆▇▅▃▃▄▅█▇▆▄
_runtime,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
_timestamp,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
_step,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
